In [1]:
import pandas as pd
import fasttext
import os
from konlpy.tag import Mecab

In [ ]:
# Mecab 토크나이저 초기화
tokenizer = Mecab(dicpath='C:/mecab/mecab-ko-dic')
# 데이터 준비
def prepare_fasttext_data(data_df, output_file, tokenizer):
    """
    FastText 형식으로 데이터를 준비합니다.
    FastText는 "__label__<class>" 형식으로 데이터를 요구합니다.
    Mecab 토크나이저를 사용하여 텍스트를 토크나이징합니다.
    """
    with open(output_file, 'w', encoding='utf-8') as f:
        for _, row in data_df.iterrows():
            label = f"__label__{row['class']}"
            # Mecab으로 토크나이징 후 공백으로 연결
            tokens = tokenizer.morphs(str(row['store']))
            tokenized_text = " ".join(tokens)
            f.write(f"{label} {tokenized_text}\n")

# 데이터 로드
data_path = './processed_data/region_all_processed_data.csv'
data_df = pd.read_csv(data_path)

# 컬럼 이름 확인 및 변경
print(data_df.columns)  # 컬럼 이름 확인
data_df.rename(columns={'상호명_Regulated': 'store', '클래스': 'class'}, inplace=True)  # 컬럼 이름 변경

# 데이터 분할
train_df = data_df.sample(frac=0.8, random_state=42)  # 80% 훈련 데이터
test_df = data_df.drop(train_df.index)  # 나머지 20% 테스트 데이터

# FastText 형식으로 데이터 준비
train_file = './processed_data/fasttext_train_tokenized.txt'
test_file = './processed_data/fasttext_test_tokenized.txt'
prepare_fasttext_data(train_df, train_file, tokenizer)
prepare_fasttext_data(test_df, test_file, tokenizer)

Index(['상가업소번호', '지역', '상호명_Regulated', '업종소분류_Regulated', '클래스'], dtype='object')


: 

In [3]:
# 모델 학습
model_path = './saved_model/fasttext/fasttext_model4.bin'
if not os.path.exists(model_path):
    model = fasttext.train_supervised(
        input=train_file,
        epoch=50,
        lr=0.5,
        wordNgrams=3,
        bucket=2000000,
        dim=200,
        loss='softmax'
    )
    model.save_model(model_path)
else:
    model = fasttext.load_model(model_path)

In [4]:
# 추론 함수
def predict_store_class(model, tokenizer, store_name):
    """
    FastText 모델을 사용하여 상호명으로 업종을 예측합니다.
    입력된 상호명은 학습 데이터와 동일하게 토크나이징되어야 합니다.
    """
    # Mecab으로 토크나이징 후 공백으로 연결
    tokens = tokenizer.morphs(str(store_name))
    tokenized_text = " ".join(tokens)
    
    prediction = model.predict(tokenized_text)
    label = prediction[0][0].replace("__label__", "")
    confidence = prediction[1][0] * 100
    return label, confidence

# 테스트
test_store = "스타벅스R리저브강남대로점"
# 추론 시에도 토크나이저를 사용해야 합니다.
predicted_class, confidence = predict_store_class(model, tokenizer, test_store)
print(f"상호명: {test_store} -> 업종: {predicted_class} (확률: {confidence:.2f}%)")

# 테스트 데이터 평가
correct = 0
total = len(test_df)
for _, row in test_df.iterrows():
    # 추론 함수에 토크나이저를 전달합니다.
    predicted_class, _ = predict_store_class(model, tokenizer, row['store'])
    if predicted_class == str(row['class']):
        correct += 1
accuracy = correct / total * 100
print(f"테스트 데이터 정확도: {accuracy:.2f}%")

상호명: 스타벅스R리저브강남대로점 -> 업종: 214 (확률: 86.17%)
테스트 데이터 정확도: 61.40%


In [5]:
# 모델 학습 (자동 튜닝)
model_path = './saved_model/fasttext/fasttext_model_autotune.bin'
if not os.path.exists(model_path):
    # autotuneValidationFile에 검증 데이터를 지정하고, autotuneDuration으로 튜닝 시간(초)을 설정합니다.
    model = fasttext.train_supervised(
        input=train_file,
        autotuneValidationFile=test_file,
        autotuneDuration=3600  # 30분 동안 자동 튜닝
    )
    model.save_model(model_path)
else:
    model = fasttext.load_model(model_path)

# 모델 평가
# model.test() 함수를 사용하면 (샘플 수, 정밀도, 재현율)을 반환합니다.
result = model.test(test_file)
print(f"테스트 데이터 정확도: {result[1] * 100:.2f}%")

테스트 데이터 정확도: 64.04%


In [9]:
# 자동 튜닝으로 저장된 모델 로드
model_path = './saved_model/fasttext/fasttext_model_autotune.bin'
model = fasttext.load_model(model_path)

# 모델 학습에 사용된 하이퍼파라미터 조회
args = model.f.getArgs()

# 결과 출력
print("자동 튜닝으로 찾은 최적 하이퍼파라미터:")
print(f"- 학습률 (lr): {args.lr}")
print(f"- 임베딩 차원 (dim): {args.dim}")
print(f"- 에포크 (epoch): {args.epoch}")
print(f"- 단어 n-gram (wordNgrams): {args.wordNgrams}")
print(f"- 손실 함수 (loss): {args.loss}")
print(f"- 버킷 크기 (bucket): {args.bucket}")

자동 튜닝으로 찾은 최적 하이퍼파라미터:
- 학습률 (lr): 0.05
- 임베딩 차원 (dim): 135
- 에포크 (epoch): 4
- 단어 n-gram (wordNgrams): 4
- 손실 함수 (loss): loss_name.softmax
- 버킷 크기 (bucket): 3408660


In [13]:
import fasttext
import os

# 사전 학습된 모델 파일 경로
pretrained_vectors_path = './saved_model/fasttext/cc.ko.300.vec' 
# 파인튜닝된 모델을 저장할 경로
finetuned_model_path = './saved_model/fasttext/fasttext_model_finetuned.bin'

# 이전에 준비한 학습 데이터 파일
train_file = './processed_data/fasttext_train_tokenized.txt'
test_file = './processed_data/fasttext_test_tokenized.txt'

if not os.path.exists(finetuned_model_path):
    print("사전 학습된 모델을 사용하여 파인튜닝을 시작합니다...")
    # train_supervised에 pretrainedVectors 인자를 추가합니다.
    # 이전에 찾은 최적의 하이퍼파라미터를 사용합니다.
    model_finetuned = fasttext.train_supervised(
        input=train_file,
        pretrainedVectors=pretrained_vectors_path,
        epoch=4,
        lr=0.05,
        wordNgrams=4,
        dim=300,  # 중요: pretrained vector의 차원(300)과 일치시켜야 합니다.
        loss='softmax'
    )
    model_finetuned.save_model(finetuned_model_path)
    print(f"파인튜닝 완료. 모델이 {finetuned_model_path}에 저장되었습니다.")
else:
    print(f"이미 파인튜닝된 모델({finetuned_model_path})을 로드합니다.")
    model_finetuned = fasttext.load_model(finetuned_model_path)

# 파인튜닝된 모델 성능 평가
print("\n파인튜닝된 모델 성능 평가:")
result_finetuned = model_finetuned.test(test_file)
print(f"테스트 데이터 정확도: {result_finetuned[1] * 100:.2f}%")

사전 학습된 모델을 사용하여 파인튜닝을 시작합니다...
파인튜닝 완료. 모델이 ./saved_model/fasttext/fasttext_model_finetuned.bin에 저장되었습니다.

파인튜닝된 모델 성능 평가:
테스트 데이터 정확도: 63.49%


In [ ]:
# 사전 학습된 모델 파일 경로
pretrained_vectors_path = './saved_model/fasttext/cc.ko.300.vec'
# 파인튜닝 + 자동튜닝된 모델을 저장할 경로
autotuned_finetuned_model_path = './saved_model/fasttext/fasttext_model_autotune_finetuned.bin'

# 데이터 파일 경로
train_file = './processed_data/fasttext_train_tokenized.txt'
test_file = './processed_data/fasttext_test_tokenized.txt'

if not os.path.exists(autotuned_finetuned_model_path):
    print("파인튜닝을 위한 자동 튜닝을 시작합니다...")
    # autotune을 실행하되, pretrainedVectors 인자를 추가합니다.
    model_autotune_finetuned = fasttext.train_supervised(
        input=train_file,
        pretrainedVectors=pretrained_vectors_path,
        autotuneValidationFile=test_file,
        autotuneDuration=3600  # 30분 동안 자동 튜닝
    )
    model_autotune_finetuned.save_model(autotuned_finetuned_model_path)
    print(f"파인튜닝 자동 튜닝 완료. 모델이 {autotuned_finetuned_model_path}에 저장되었습니다.")
else:
    print(f"이미 자동 튜닝된 파인튜닝 모델({autotuned_finetuned_model_path})을 로드합니다.")
    model_autotune_finetuned = fasttext.load_model(autotuned_finetuned_model_path)

# 최종 모델 성능 평가
print("\n자동 튜닝된 파인튜닝 모델 성능 평가:")
result = model_autotune_finetuned.test(test_file)
print(f"테스트 데이터 정확도: {result[1] * 100:.2f}%")

# 최적 하이퍼파라미터 확인
print("\n파인튜닝을 위해 찾은 최적 하이퍼파라미터:")
args = model_autotune_finetuned.f.getArgs()
print(f"- 학습률 (lr): {args.lr}")
print(f"- 에포크 (epoch): {args.epoch}")
print(f"- 단어 n-gram (wordNgrams): {args.wordNgrams}")
print(f"- 손실 함수 (loss): {args.loss}")

파인튜닝을 위한 자동 튜닝을 시작합니다...
